<a href="https://colab.research.google.com/github/Horcruxno13/Multi-Class-Text-Classification-using-BERT-and-PyTorch/blob/main/ECommerce_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ecom_df = pd.read_csv('/content/drive/My Drive/Datasets/ecommerceCleaned.csv')

In [ ]:
X = ecom_df["Description"].values
y = ecom_df["Labels"].values

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
#import locale
#locale.getpreferredencoding = lambda: "UTF-8"
#!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.8 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
input_ids = []
for line in X:
    line = line[:512]
    encoded = tokenizer.encode(line, add_special_tokens = True)
    input_ids.append(encoded)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = 512 
input_ids = pad_sequences(input_ids, maxlen=MAX_LENGTH, dtype="long", value=0, truncating="post", padding="post")


In [ ]:
attention_masks = []
for line in input_ids:
    mask = [int(token_id > 0) for token_id in line]
    attention_masks.append(mask)

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
input_train, input_remain, label_train, label_remain = train_test_split(input_ids, y, test_size=0.3, random_state=42)
input_valid, input_test, label_valid, label_test = train_test_split(input_remain, label_remain, test_size=0.5, random_state=42)

mask_train, mask_remain, _, _ = train_test_split(attention_masks, y, test_size=0.3, random_state=42)
mask_valid, mask_test, _, _ = train_test_split(mask_remain, label_remain, test_size=0.5, random_state=42)

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
batch_size = 16

trainset = TensorDataset(torch.tensor(input_train), torch.tensor(label_train), torch.tensor(mask_train))
validset = TensorDataset(torch.tensor(input_valid), torch.tensor(label_valid), torch.tensor(mask_valid))
testset = TensorDataset(torch.tensor(input_test), torch.tensor(label_test), torch.tensor(mask_test))

trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size, num_workers = 2, pin_memory = True)
valloader = DataLoader(validset, shuffle=True, batch_size=batch_size, num_workers = 1)
testloader = DataLoader(testset, shuffle=True, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 4,  output_attentions = False, output_hidden_states = False)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored

In [ ]:
optimizer = AdamW(model.parameters(), lr = 2e-3, eps = 1e-8)
criterion = nn.CrossEntropyLoss()

In [ ]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def calcuate_accu(big_idx, targets):
  n_correct = (big_idx==targets).sum().item()
  return n_correct

In [ ]:
scaler = torch.cuda.amp.GradScaler()

In [ ]:
epochs = 4
epochloop = tqdm(range(epochs), position=0, desc='Training', leave=True)

for e in epochloop:
  model.train()
  train_loss = 0
  train_acc = 0

  for id, batch in enumerate(trainloader):
    inputids = batch[0].to(device)
    inputlabels = batch[1].to(device)
    inputmasks = batch[2].to(device)

    for param in model.parameters():
      param.grad = None
    outputs = model(inputids, token_type_ids=None, attention_mask = inputmasks, labels = inputlabels)
    print(id)
    loss = outputs[0]
    train_loss = train_loss + loss.item()
    #equals = outputs == inputlabels
    #acc = torch.mean(torch.FloatTensor(equals))
    #train_acc = train_acc + acc.item()
    #loss = criterion(outputs.squeeze(), labels.float())
    scaler.scale(loss).backward()
    print("Loss: ", train_loss)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    del inputids, inputlabels, inputmasks

  history['train_loss'].append(train_loss / len(trainloader))
  #history['train_acc'].append(train_acc / len(trainloader))
  print("Training Loss", history['train_loss'])
  #print("Training Accuracy", history['train_acc'])

  model.eval()
  val_loss = 0
  val_acc = 0
  nb_eval_steps = 0

  for batch in valloader:
    batch = tuple(t.to(device) for t in batch)
    inputids = batch[0]
    inputlabels = batch[1]
    inputmasks = batch[2]

    with torch.no_grad():
      outputs = model(inputids, token_type_ids=None, attention_mask = inputmasks)
      #equals = outputs == inputlabels
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = inputlabels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      #acc = torch.mean(equals.type(torch.FloatTensor))
      #val_acc = val_acc + acc.item()
      #loss = criterion(outputs.squeeze(), labels.float())
      #val_loss = val_loss + loss.item()
      nb_eval_steps += 1

  
  #history['val_loss'].append(val_loss / len(valloader))
  history['val_acc'].append(val_acc / nb_eval_steps)
  #print("Validation Loss", history['val_loss'])
  print("Validation Accuracy", history['val_acc'])